In [1]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [2]:
data = pd.read_csv("movie.csv")
data.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


# Data preprocessing

Cleaning text data (removing punctuation, turning to lowercase, removing stop words)

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nehue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nehue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    words = word_tokenize(text)
    text = ' '.join(word for word in words if word not in stop_words)
    return text


In [5]:
data['text'] = data['text'].apply(preprocess_text)

In [6]:
data.head()

,text,label
0,grew b 1965 watching loving thunderbirds mates...,0
1,put movie dvd player sat coke chips expectatio...,0
2,people know particular time past like feel nee...,0
3,even though great interest biblical movies bor...,0
4,im die hard dads army fan nothing ever change ...,1


# Feature engineering
Converting each sample's text into numerical features that can be used as input for an ML model

## Feature Extraction Technique: **TF-IDF**
**Term frequency - inverse document frecuency**, which highlights the importance of words in a sample in contrast to the collection of all samples, giving higher weights to terms that are frequent within a text sample but rare across the rest of samples

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use TF-IDF vectorizer to convert text data to numerical features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['text'])
y = data['label']

In [8]:
X

<40000x159207 sparse matrix of type '<class 'numpy.float64'>'
	with 4024949 stored elements in Compressed Sparse Row format>

In [9]:
y.head()

0    0
1    0
2    0
3    0
4    1
Name: label, dtype: int64

## Choosing the right model

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
models = {
    "Logistic Regression": make_pipeline(LogisticRegression(max_iter=1000)),
    "Naive Bayes": make_pipeline(MultinomialNB()),
    "SVM": make_pipeline(SVC(kernel='linear', probability=True)),
    "Random Forest": make_pipeline(RandomForestClassifier(n_estimators=100))
}

In [14]:
def evaluate_models(models, X, y):
    results = []
    
    for model_name, model in models.items():
        scores = cross_validate(model, X, y, scoring=['accuracy', 'precision', 'recall', 'f1'], cv=4, n_jobs=-1)
        results.append({
            'Model': model_name,
            'Accuracy': np.mean(scores['test_accuracy']),
            'Precision': np.mean(scores['test_precision']),
            'Recall': np.mean(scores['test_recall']),
            'F1 Score': np.mean(scores['test_f1'])
        })
    
    return pd.DataFrame(results)

results = evaluate_models(models, X_train, y_train)
print(results)

KeyboardInterrupt: 

### Best model: **SVM**

## Finding the correct hyperparameters with Grid Search

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear','rbf', 'poly', 'sigmoid']
}

grid_search = GridSearchCV(SVC(), param_grid, refit=True, verbose=1, cv=3, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 64 candidates, totalling 192 fits


In [ ]:
model = grid_search.best_estimator_

## Evaluating the model

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y_test, y_pred))


Accuracy: 0.90
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      3966
           1       0.89      0.91      0.90      4034

    accuracy                           0.90      8000
   macro avg       0.90      0.90      0.90      8000
weighted avg       0.90      0.90      0.90      8000



## Predicting with new reviews

In [ ]:
new_reviews = [
    "The plot was kind of good I must admit, and loved the action scenes",
    "The movies was great... For a 2 year old, who doesn't differentiate a car from a plane",
]

new_reviews = [preprocess_text(review) for review in new_reviews]

X_new = vectorizer.transform(new_reviews)

predictions = model.predict(X_new)
print(predictions)


[1 1]
